In [10]:
from datetime import datetime, timedelta
from flights_feast_module.feast_repo import airline, arr_airport, dep_airport, flight_stats_view
from feast import FeatureStore, RepoConfig
import pytz

fs = FeatureStore(repo_path="./flights_feast_module")

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
#bigQuery client

import google.auth
from google.cloud import bigquery

credentials, project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

bq_client = bigquery.Client(credentials=credentials, project=project_id,)


In [12]:
# get 10 samples from historical flight records
query="""
SELECT airline,departure_airport, arrival_airport, departure_delay, event_timestamp
 FROM `red-beryl-labs.redbq.flight_records` order by rand() limit 10
"""
flight_records = bq_client.query(query).to_dataframe()

num_of_rows = flight_records.shape[0]
print('rows:',num_of_rows)
flight_records.head(n=5)

rows: 10


,airline,departure_airport,arrival_airport,departure_delay,event_timestamp
0,DL,IAH,ATL,-7.0,2004-01-27 17:15:00+00:00
1,US,PHX,LAS,-6.0,2010-08-04 10:20:00+00:00
2,OH,CVG,LGA,0.0,2009-10-22 07:00:00+00:00
3,WN,LAS,SLC,5.0,2006-12-03 10:30:00+00:00
4,FL,ATL,MSY,-4.0,2011-10-17 10:25:00+00:00


In [13]:
#generate a sample flight records
import numpy as np
import pandas as pd

flight_records['event_timestamp'] = pd.to_datetime(np.random.randint(datetime(2011, 10, 18).timestamp(),datetime(2011, 10, 20).timestamp(), size=10), unit='s', utc=True)

flight_records.head(n=5)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,airline,departure_airport,arrival_airport,departure_delay,event_timestamp
0,DL,IAH,ATL,-7.0,2011-10-18 13:25:54+00:00
1,US,PHX,LAS,-6.0,2011-10-18 15:02:46+00:00
2,OH,CVG,LGA,0.0,2011-10-18 22:01:22+00:00
3,WN,LAS,SLC,5.0,2011-10-18 12:25:05+00:00
4,FL,ATL,MSY,-4.0,2011-10-19 19:48:58+00:00


In [14]:
#this is a feast's big query job
job = fs.get_historical_features(feature_refs=[
            "flights_stats:avg_departure_delay",
            "flights_stats:avg_arrival_delay"
        ],                         
        entity_df=flight_records)

print("big query job {} started, please hold".format(type(job)))
train_data = job.to_df()
print("big query job completed")


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


big query job <class 'feast.infra.offline_stores.bigquery.BigQueryRetrievalJob'> started, please hold
big query job completed


In [15]:
num_of_rows = train_data.shape[0]
print('rows:',num_of_rows)
train_data.head()

rows: 10


,airline,departure_airport,arrival_airport,departure_delay,event_timestamp,flights_stats__avg_departure_delay,flights_stats__avg_arrival_delay
0,OH,CVG,LGA,0.0,2011-10-18 22:01:22+00:00,NaN,NaN
1,OO,BOI,SLC,0.0,2011-10-19 20:48:47+00:00,1.24,-1.96
2,WN,IND,MSY,0.0,2011-10-19 08:11:32+00:00,NaN,NaN
3,FL,ATL,MSY,-4.0,2011-10-19 19:48:58+00:00,11.29,14.29
4,WN,LAS,SLC,5.0,2011-10-18 12:25:05+00:00,6.49,3.44


In [16]:
#store data as historical retrival from data warehouse
def store_feature_data(dataset, table, data):
    reference = bq_client.dataset(dataset_id)
    dataset = bq_client.get_dataset(reference)
    print(dataset)

    job_config = bigquery.job.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

    job = bq_client.load_table_from_dataframe(data, table, job_config=job_config)

    print("BigQuery job started, please hold for result")
    job.result()


    #if not dataset.exists():
    #dataset.create()
    print('Loaded {} rows into Big Query{}:{}.'.format(
            job.output_rows, dataset_id, table))

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
dataset_id="redbq"
print("BigQuery jobs started, please hold...")
#store flights retrieved
store_feature_data(dataset_id,"redbq.ml_flight_train", train_data)
print("BigQuery jobs completed.")

BigQuery jobs started, please hold...
Dataset(DatasetReference('red-beryl-labs', 'redbq'))
BigQuery job started, please hold for result
Loaded 10 rows into Big Queryredbq:redbq.ml_flight_train.
BigQuery jobs completed.
